In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("/Users/issackondreddy/Desktop/Education/Projects/Cancer Detection/Docs/Level_2_MA_Selection.xlsx")
for function in df['Benchmark Function'].unique():
    func_df = df[df['Benchmark Function'] == function]
    for col in ['Best Fitness', 'Average Fitness', 'Standard Deviation']:
        min_val = func_df[col].min()
        max_val = func_df[col].max()
        df.loc[func_df.index, col + ' Norm'] = (func_df[col] - min_val) / (max_val - min_val)
    # Invert 'Convergence Generation'
    min_val = func_df['Convergence Generation'].min()
    max_val = func_df['Convergence Generation'].max()
    df.loc[func_df.index, 'Convergence Generation Norm'] = 1 - (func_df['Convergence Generation'] - min_val) / (max_val - min_val)

# Rank the normalized values
for col in ['Best Fitness Norm', 'Average Fitness Norm', 'Standard Deviation Norm', 'Convergence Generation Norm']:
    df[col + ' Rank'] = df.groupby('Benchmark Function')[col].rank()

# Calculate weighted score with your weights
df['Weighted Score'] = (
    df['Best Fitness Norm Rank'] * 0.5 + 
    df['Average Fitness Norm Rank'] * 0.3 + 
    df['Convergence Generation Norm Rank'] * 0.1 +
    df['Standard Deviation Norm Rank'] * 0.1
)

# Sort and select top MAs
top_mAs = df.sort_values(by='Weighted Score').groupby('Benchmark Function').head(5)


In [3]:
df.head()

,Algorithms,Benchmark Function,Best Fitness,Average Fitness,Standard Deviation,Convergence Generation,Best Fitness Norm,Average Fitness Norm,Standard Deviation Norm,Convergence Generation Norm,Best Fitness Norm Rank,Average Fitness Norm Rank,Standard Deviation Norm Rank,Convergence Generation Norm Rank,Weighted Score
0,Genetic Algorithm -> Salp Swarm Algorithm,Sphere,3.087797e+03,4.731199e+03,6.656306e+03,52,0.106846,0.0,0.00000,0.250000,2.0,1.0,1.0,2.0,1.6
1,NaN,Ackley,1.864000e+01,1.925000e+01,6.400000e-01,66,0.000000,0.0,1.00000,0.000000,1.0,1.0,3.0,1.0,1.2
2,NaN,Rastrigin,4.394395e+03,5.716790e+03,6.554213e+03,60,0.587154,0.0,0.00000,0.283951,2.0,1.0,1.0,2.0,1.6
3,NaN,Rosenbrock,2.914422e+07,3.519801e+08,2.551884e+09,67,0.000000,0.0,0.00000,0.344086,1.0,1.0,1.0,2.0,1.1
4,Genetic Algorithm → Differential Evolution,Sphere,1.335137e+04,2.190882e+04,1.233447e+04,4,1.000000,1.0,0.43071,1.000000,3.0,3.0,2.0,3.0,2.9


In [5]:
output_file_path = "/Users/issackondreddy/Desktop/Education/Projects/Cancer Detection/Docs/Level_2_MA_Selection.xlsx"

# Save the DataFrame to a CSV file
df.to_excel(output_file_path, index=False)

In [6]:
clean_df = df.dropna(subset=['Algorithms', 'Weighted Score']).copy()

# Calculate the min and max for normalization
min_score = clean_df['Weighted Score'].min()
max_score = clean_df['Weighted Score'].max()

# Use .loc to avoid SettingWithCopyWarning
clean_df.loc[:, 'Normalized Weighted Score'] = (clean_df['Weighted Score'] - min_score) / (max_score - min_score)

# Sort by 'Normalized Weighted Score' in ascending order to get the best scores first
sorted_clean_df = clean_df.sort_values(by='Normalized Weighted Score', ascending=True)

# Drop duplicate 'Algorithm' entries, keeping the first occurrence
top_unique_MAs = sorted_clean_df.drop_duplicates(subset='Algorithms')

# Select the top 3 MAs with the lowest normalized weighted score
top_3_MAs = top_unique_MAs.head(3)

# Display the top 3 MAs
print(top_3_MAs[['Algorithms', 'Benchmark Function', 'Normalized Weighted Score']])

                                      Algorithms Benchmark Function  \
8  Salp Swarm Algorithm → Differential Evolution             Sphere   
0     Genetic Algorithm -> Salp Swarm Algorithm              Sphere   
4     Genetic Algorithm → Differential Evolution             Sphere   

   Normalized Weighted Score  
8                   0.000000  
0                   0.071429  
4                   1.000000  
